# 2.4.3 Compare Group individual models Version 2 with format updated

In [1]:
import numpy as np
import pandas as pd
import torch
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [2]:
import sys
import numpy as np
from numpy.random import seed
### imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Turn off TensorFlow logging
import tensorflow as tf


import numpy as np
import pandas as pd
import random
#from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
import loadfile
import addons

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Input, add
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import load_model, save_model, Model


from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices
from utils import *
from model import *


tf.__version__

'2.3.0'

# General Settings

In [3]:

shimmer_global_mean = [-0.012359981,-0.0051663737,0.011612018,
                        0.05796114,0.1477952,-0.034395125 ]

shimmer_global_stddev = [0.05756385,0.040893298,0.043825723,
                        17.199743,15.311142,21.229317 ]

shimmer_trended_mean = [-0.000002,-0.000002,-0.000000,
                0.058144,0.147621,-0.033260 ]

shimmer_trended_stddev = [0.037592,0.034135,0.032263,
                17.209038,15.321441,21.242532 ]

all_zero_means = [0,0,0,0,0,0]

meanvals = all_zero_means
stdvals = shimmer_trended_stddev


random_seed  = 1000
seed(random_seed)
tf.random.set_seed(random_seed)
np.random.seed(random_seed)



In [4]:
def seed_everything(seed= 1000):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(seed= random_seed)  

In [5]:
def acti_model(input_shape):
        
        model = Sequential()
        
        model.add(Conv1D(10, 44, strides=2,activation='relu', input_shape=input_shape))
        model.add(Conv1D(10, 20, strides=2, activation='relu',kernel_regularizer=tf.keras.regularizers.l1(0.01)))
        model.add(Conv1D(10, 4, strides=2, activation='relu',kernel_regularizer=tf.keras.regularizers.l1(0.01)))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(200, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])
        return model


def residual_module(layer_in, n_filters):
            merge_input = layer_in
            # check if the number of filters needs to be increase, assumes channels last format
            if layer_in.shape[-1] != n_filters:
                merge_input = Conv1D(n_filters, (1,), padding='same', kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l1(0.01))(layer_in)
            # conv1
            # L2 for avoiding overfitting
            conv1 = Conv1D(n_filters, (5,), padding='same', kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l2(0.01))(layer_in)
            # Batch normal for re-scaling data and better fit data
            conv1 = tf.keras.layers.BatchNormalization(1)(conv1)
            conv1 = tf.keras.layers.ReLU()(conv1)
            # conv2
            conv2 = Conv1D(n_filters, (5,), padding='same', kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l2(0.01))(conv1)
            conv2 = tf.keras.layers.BatchNormalization(1)(conv2)
            
            # add filters, assumes filters/channels last
            #out = conv2
            out = add([conv2, merge_input])
            # activation function
            layer_out = Activation('relu')(out)
            return layer_out
        

def res_model(input_shape):
            in_layer = Input(shape = input_shape)
            out = Conv1D(10, 40, strides=2)(in_layer)
            out = residual_module(layer_in= out,  n_filters=10)
            out = residual_module(layer_in= out,  n_filters=10)
            out = GlobalAveragePooling1D()(out)
            # L1 for feature selection
            out = Dense(200, activation='relu',kernel_regularizer=tf.keras.regularizers.l1(0.01))(out)
            out = Dense(1, activation='sigmoid')(out)
            model = Model(inputs=in_layer, outputs=out)
            return model
        
def transfer_model(input_shape, tf_model):
            in_layer = Input(shape = input_shape)
            out = tf_model(in_layer)
            out = Dense(1, activation='sigmoid')(out)
            model = Model(inputs=in_layer, outputs=out)
            return model
    
    
    
def transfer_model_v2(input_shape, tf_model):
            in_layer = Input(shape = input_shape)
            
            model = Sequential()
            model.add(in_layer)
            for layer in tf_model.layers[1:-1]:  # 跳过最后一层 
                model.add(layer)
            model.add(Dense(1, activation='sigmoid'))
            return model
    
# newmodel = my_model(input_shape =(900,6) )

# newmodel.compile(loss='binary_crossentropy',
#                 optimizer='adam', metrics=['accuracy'])


In [11]:

from dataset import create_train_test_file_list,  balance_data_indices  #Person_MealsDataset,
from utils import *
from model import *
def train_models(model, win_ls = [],EPOCHS = 10,stridesec = 1,name = "wenkanw",model_name="v2" ,random_seed= 1000, split_day=False,test_balanced=False):
    
    from datetime  import datetime
    batch_size = 128
    outfile = sys.stdout
    perf = {"model":[],"win(sec)":[], "acc":[],"recall":[], "auc":[]}
    model_ls = []
    hist_ls = []
    seed_everything(seed= random_seed) 
    for winsize in win_ls:
        winmin = winsize
        winlength = int(winmin * 60 * 15)
        step = int(stridesec * 15)
        start_time = datetime.now()
        arr = ["echo -n 'PBS: node is '; cat $PBS_NODEFILE",\
              "echo PBS: job identifier is $PBS_JOBID",\
              "echo PBS: job name is $PBS_JOBNAME"]
        [os.system(cmd) for cmd in arr]
        print("*****************************************************************\n", file=outfile, flush=True)
        print("Execution Started at " + start_time.strftime("%m/%d/%Y, %H:%M:%S"), file=outfile, flush=True)
        print("WindowLength: {:.2f} min ({:d} datum)\tSlide: {:d} ({:d} datum)\tEpochs:{:d}\n".format(winmin, winlength, stridesec, step, EPOCHS), file=outfile, flush=True)


        pathtemp = "../models/" + name+"_models" +"/"+model_name+"_M_F_"
        #pathtemp = "../models/" + name +"/"+model_name+"_M_F_"
        modelpath = pathtemp + "{:f}Min.h5".format(winmin)
        jsonpath = pathtemp + "{:f}Min.json".format(winmin)
        print("Model to Save: ",modelpath)
        print()
        # Load the dataset
        
        person = name
        if split_day:
            pathtemp = "../models/" + name+"_models" +"/"+model_name+"_split_day_M_F_"
            modelpath = pathtemp + "{:f}Min.h5".format(winmin)
            jsonpath = pathtemp + "{:f}Min.json".format(winmin)
            create_train_test_file_list(file_name= "all_files_list.txt",person_name =name,
                         out_path = "../data-file-indices/",root_path= "../",
                         test_ratio = 0.2, print_flag = True, shuffle=True, random_state=random_seed)

            meal_data_train = Person_MealsDataset(person_name= person, file_name = "train_files", winmin = winmin,stridesec = stridesec)
            meal_data_test = Person_MealsDataset(person_name= person, file_name = "test_files", winmin = winmin,stridesec = stridesec)

            train_indices, valid_indices = split_train_test_indices(X= [i for i in range(len(meal_data_train.labels))],
                                                                    y = meal_data_train.labels, test_size = 0.2,
                                                                   random_seed = random_seed)
            #balanced train set
            trainset_labels = meal_data_train.labels[train_indices]
            train_indices = balance_data_indices(trainset_labels,data_indices= train_indices,mode="under", shuffle=True,random_state = random_seed,replace= False)

            # balance test set
            testset_labels = meal_data_test.labels
            if test_balanced:
                test_indices = balance_data_indices(testset_labels,data_indices=[i for i in range(len(meal_data_test))] ,mode="under", shuffle=True,random_state = random_seed,replace= False)
            else:
                # without balancing data
                test_indices = [i for i in range(len(meal_data_test))] 
            # get numpy dataset
            balancedData, balancedLabels = meal_data_train.get_subset(train_indices)
            valid_balancedData, valid_balancedLabels = meal_data_train.get_subset(valid_indices)
            test_Data, test_Labels = meal_data_test.get_subset(test_indices)

        else:
        
            meal_data = Person_MealsDataset(person_name= person, file_name = "all_files_list", winmin = winmin,stridesec = stridesec)
            samples,labels =  meal_data.data_indices, meal_data.labels
            # split train set and test set
            train_indices, test_indices = split_train_test_indices(X= [i for i in range(len(labels))],
                                                                    y = labels, test_size = 0.2,
                                                                   random_seed = random_seed)
            # balance train set
            trainset_labels = labels[train_indices]
            train_indices_balanced = balance_data_indices(trainset_labels,data_indices= train_indices,mode="under", shuffle=True,random_state = random_seed,replace= False)
            #balance test set
            testset_labels = labels[test_indices]
            if test_balanced:
                test_indices = balance_data_indices(testset_labels,data_indices= test_indices,mode="under", shuffle=True,random_state = random_seed,replace= False)
            else:
                test_indices = [i for i in range(len(meal_data_test))] 
            train_set_balanced = torch.utils.data.Subset(meal_data, train_indices_balanced)
            test_set = torch.utils.data.Subset(meal_data, test_indices)

            train_loader = torch.utils.data.DataLoader(train_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)
            test_loader = torch.utils.data.DataLoader(test_set ,batch_size=batch_size, shuffle=True,num_workers=2)

            print("Data Loader Created")            
            
            # split validation set
            balanced_trainset_labels = labels[train_indices_balanced]
            train_indices, valid_indices = split_train_test_indices(X= train_indices_balanced,
                                                                    y = balanced_trainset_labels, test_size = 0.2,
                                                                   random_seed = random_seed)
            valid_set_balanced = torch.utils.data.Subset(meal_data, valid_indices)
            valid_loader = torch.utils.data.DataLoader(valid_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)

            balancedData, balancedLabels = meal_data.get_subset(train_indices)
            valid_balancedData, valid_balancedLabels = meal_data.get_subset(valid_indices)
            test_Data, test_Labels = meal_data.get_subset(test_indices)
        

        #training settings
        mcp_save = tf.keras.callbacks.ModelCheckpoint(modelpath, save_best_only=True, monitor='accuracy')
        

        scheduler = tf.keras.callbacks.ReduceLROnPlateau( monitor='val_loss', factor=0.1, patience=3, verbose=0,
                                             mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.)
        #train model
        H = model.fit(x=balancedData, y = balancedLabels,
                       validation_data=(valid_balancedData, valid_balancedLabels),
                    epochs = EPOCHS, batch_size=batch_size, verbose=1,
                    callbacks=[mcp_save,scheduler]) # removed addons.LossHistory(jsonpath) for compatibility with TensorFlow 2.2.0, needs to be re-added at some point

        print("Max value: ", max(H.history['accuracy']), " at epoch", np.argmax(H.history['accuracy']) + 1)

        from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, balanced_accuracy_score, f1_score
        predictions = model.predict(x=test_Data)
        threshold = 0.5
        wacc =  balanced_accuracy_score(test_Labels,predictions>=threshold)
        f1 =  f1_score(test_Labels,predictions>=threshold)
        
        acc =  accuracy_score(test_Labels,predictions>=threshold)
        recall = recall_score(test_Labels,predictions>=threshold)
        auc = roc_auc_score(test_Labels,predictions>=threshold)
        print("Test Accuracy:", acc)
        print("Recall Accuracy:", recall)
        print("AUC Score:", auc)

        perf["model"].append("ActiModel")
        perf["win(sec)"].append(winmin*60)
        perf["acc"].append(acc)
        perf["recall"].append(recall)
        perf["auc"].append(auc)
        model_ls.append(model)
        hist_ls.append(H)
    perf_df = pd.DataFrame(perf)
    print(perf_df)
    return perf_df, model_ls, hist_ls




In [12]:
# name = "lawler"
# group_model_W  = tf.keras.models.load_model('../models/CAD_models/resnet_M_F_1.000000Min.h5')
# group_model_S  = tf.keras.models.load_model('../models/ActiModels/CAD_M_F_1.000000Min.h5')
# individual_model = tf.keras.models.load_model('../models/'+ name+ '_models/resnet_M_F_1.000000Min.h5')
# individual_model.trainable


In [13]:
# individual_model.get_weights()
# individual_model.layers.pop(0)
# individual_model.layers.pop(-1)
# individual_model.layers
# # config = individual_model._layers[-1].get_config()
# # config

In [14]:
# individual_model.summary()

In [15]:
# # config = individual_model._layers[-1].get_config()
# # new_layer = Dense.from_config(config)
# new_layer= Dense(1,activation="sigmoid")
# individual_model._layers[-1] =new_layer
# individual_model.summary()
# individual_model.compile(loss='binary_crossentropy',
#                 optimizer='adam', metrics=['accuracy'])


In [16]:
# model = Sequential()
# individual_model.layers.pop(-1)
# individual_model._layers.pop(-1)
# model.add(individual_model)
# model.add(Dense(1, activation="sigmoid",name="dense_7"))
# model.summary()

In [17]:
# name ='lawler'
# winmin = 1
# win_size = 15*winmin*60
# # in_layer = Input(shape = (win_size,6), name="input_4")
# # individual_model.layers.pop(0)
# individual_model.layers.pop(-1)
# # individual_model._layers.pop(0)
# individual_model._layers.pop(-1)
# out = individual_model(in_layer)
# out = Dense(1, activation="sigmoid",name="dense_7")(out)
# model = Model(individual_model.input , out)
# model.summary()
# name = "lawler"
# individual_model = tf.keras.models.load_model('../models/'+ name+ '_models/resnet_M_F_1.000000Min.h5')
# individual_model.trainable =True
# new_model = individual_model
# for layer in [new_model.layers[-1]]:
#     if hasattr(layer,'init'):
#         input_dim = layer.input_shape[1]
#         new_weights = layer.init((input_dim, layer.output_dim),name='{}_W'.format(layer.name))
#         layer.trainable_weights[0].set_value(new_weights.get_value())
        
# new_model.compile(loss='binary_crossentropy',
#                 optimizer='adam', metrics=['accuracy'])

        
# perf_df_1, model_ls_1, hist_ls_1 = train_models(new_model,win_ls = [winmin],EPOCHS = 30,stridesec = 5,model_name="resnet_1min_transferCAD",name = 'lawler',split_day=True)

# Train single models

In [167]:
winmin= 6
win_size = 15*winmin*60
model_1 = acti_model(input_shape =(win_size,6) )
model_1.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])
perf_df_1, model_ls_1, hist_ls_1 = train_models(model_1,win_ls = [winmin],EPOCHS = 30,stridesec = 5,model_name="acti_6min",name = "lawler",split_day=True)


*****************************************************************

Execution Started at 01/08/2021, 18:56:49
WindowLength: 6.00 min (5400 datum)	Slide: 5 (75 datum)	Epochs:30

Model to Save:  ../models/lawler_models/acti_6min_M_F_6.000000Min.h5

Train:
IndividualData/lawler-data/10.8/10.8.shm
IndividualData/lawler-data/11.15/11.15.shm
IndividualData/lawler-data/10.14/10.14.shm
IndividualData/lawler-data/9.23/9.23_13hr.shm
IndividualData/lawler-data/10.12/10.12.shm
IndividualData/lawler-data/11.11/11.11.shm
IndividualData/lawler-data/11.4/11.4.shm
IndividualData/lawler-data/9.25/9.25_1-46.shm
IndividualData/lawler-data/10.6/10.6.shm
IndividualData/lawler-data/10.4/10.4.shm
IndividualData/lawler-data/10.28/10.28.shm
IndividualData/lawler-data/11.3/11.3.shm
IndividualData/lawler-data/10.20/evening_2hr_20min/10.20.shm
IndividualData/lawler-data/10.26/10.26.shm
IndividualData/lawler-data/10.31/10.31.shm
IndividualData/lawler-data/10.31/evening-2hr_goodDinnerTemplate_CFAmeal/10.31.shm
Indivi

KeyboardInterrupt: 

In [134]:
winmin= 1
win_size = 15*winmin*60
model_1 = res_model(input_shape =(900,6) )
model_1.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])
perf_df_1, model_ls_1, hist_ls_1 = train_models(model_1,win_ls = [1],EPOCHS = 30,stridesec = 5,model_name="resnet",name = "wenkanw",split_day=True)


*****************************************************************

Execution Started at 01/07/2021, 11:53:59
WindowLength: 1.00 min (900 datum)	Slide: 5 (75 datum)	Epochs:30

Model to Save:  ../models/wenkanw_models/resnet_M_F_1.000000Min.h5

Train:
IndividualData/wenkanw-data/11-21-20/Dinner/Dinner.shm
IndividualData/wenkanw-data/9-17-20/9-17-20.shm
IndividualData/wenkanw-data/9-29-20/9-29-20.shm
IndividualData/wenkanw-data/11-22-20/Lunch/Lunch.shm
IndividualData/wenkanw-data/11-21-20/lunch/lunch.shm
IndividualData/wenkanw-data/10-9-20/10-9-20.shm
IndividualData/wenkanw-data/11-18-20/11-18-20.shm
IndividualData/wenkanw-data/10-14-20/10-14-20.shm
IndividualData/wenkanw-data/10-12-20/10-12-20.shm
IndividualData/wenkanw-data/11-16-20/11-16-20.shm
IndividualData/wenkanw-data/9-27-20/9-27-20.shm
IndividualData/wenkanw-data/10-3-20/10-3-20.shm
IndividualData/wenkanw-data/10-10-20/10-10-20.shm
IndividualData/wenkanw-data/11-22-20/Dinner/Dinner.shm

test: 
IndividualData/wenkanw-data/11-14-20

In [135]:
winmin= 1
win_size = 15*winmin*60
model_1 = res_model(input_shape =(900,6) )
model_1.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])
perf_df_2, model_ls_2, hist_ls_2 = train_models(model_1,win_ls = [1],EPOCHS = 30,stridesec = 5,model_name="resnet",name = "adam",split_day=True)


*****************************************************************

Execution Started at 01/07/2021, 11:57:14
WindowLength: 1.00 min (900 datum)	Slide: 5 (75 datum)	Epochs:30

Model to Save:  ../models/adam_models/resnet_M_F_1.000000Min.h5

Train:
IndividualData/adam-data/12-02-2020/12-02-2020.shm
IndividualData/adam-data/12-10-2020/12-10-2020.shm
IndividualData/adam-data/09-22-2020/09-22-2020.shm
IndividualData/adam-data/11-25-2020/11-25-2020.shm
IndividualData/adam-data/11-12-2020/11-12-2020.shm
IndividualData/adam-data/11-26-2020/11-26-2020.shm
IndividualData/adam-data/11-17-2020/11-17-2020.shm
IndividualData/adam-data/10-28-2020/10-28-2020.shm
IndividualData/adam-data/12-07-2020/12-07-2020.shm
IndividualData/adam-data/12-04-2020/12-04-2020.shm
IndividualData/adam-data/12-03-2020/12-03-2020.shm
IndividualData/adam-data/11-11-2020/11-11-2020-1.shm
IndividualData/adam-data/12-11-2020/12-11-2020.shm
IndividualData/adam-data/11-16-2020/11-16-2020.shm
IndividualData/adam-data/11-09-2020/1

In [136]:
winmin= 1
win_size = 15*winmin*60
model_1 = res_model(input_shape =(900,6) )
model_1.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])
perf_df_3, model_ls_3, hist_ls_3 = train_models(model_1,win_ls = [1],EPOCHS = 30,stridesec = 5,model_name="resnet",name = "lawler",split_day=True)


*****************************************************************

Execution Started at 01/07/2021, 11:59:55
WindowLength: 1.00 min (900 datum)	Slide: 5 (75 datum)	Epochs:30

Model to Save:  ../models/lawler_models/resnet_M_F_1.000000Min.h5

Train:
IndividualData/lawler-data/10.8/10.8.shm
IndividualData/lawler-data/11.15/11.15.shm
IndividualData/lawler-data/10.14/10.14.shm
IndividualData/lawler-data/9.23/9.23_13hr.shm
IndividualData/lawler-data/10.12/10.12.shm
IndividualData/lawler-data/11.11/11.11.shm
IndividualData/lawler-data/11.4/11.4.shm
IndividualData/lawler-data/9.25/9.25_1-46.shm
IndividualData/lawler-data/10.6/10.6.shm
IndividualData/lawler-data/10.4/10.4.shm
IndividualData/lawler-data/10.28/10.28.shm
IndividualData/lawler-data/11.3/11.3.shm
IndividualData/lawler-data/10.20/evening_2hr_20min/10.20.shm
IndividualData/lawler-data/10.26/10.26.shm
IndividualData/lawler-data/10.31/10.31.shm
IndividualData/lawler-data/10.31/evening-2hr_goodDinnerTemplate_CFAmeal/10.31.shm
Individual

In [159]:
winmin= 1
win_size = 15*winmin*60
model_1 = res_model(input_shape =(900,6) )
model_1.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])
perf_df_4, model_ls_4, hist_ls_4 = train_models(model_1,win_ls = [1],EPOCHS = 30,stridesec = 5,model_name="resnet",name = "shaurya",split_day=True)


*****************************************************************

Execution Started at 01/07/2021, 13:43:55
WindowLength: 1.00 min (900 datum)	Slide: 5 (75 datum)	Epochs:30

Model to Save:  ../models/shaurya_models/resnet_M_F_1.000000Min.h5

Train:
IndividualData/shaurya-data/10.22.2020/Data.shm
IndividualData/shaurya-data/11.15.2020/Data.shm
IndividualData/shaurya-data/11.17.2020/Data.shm
IndividualData/shaurya-data/11.1.2020/Data.shm
IndividualData/shaurya-data/11.18.2020/Data.shm

test: 
IndividualData/shaurya-data/10.25.2020/Data.shm

All files: 
IndividualData/shaurya-data/10.25.2020/Data.shm
IndividualData/shaurya-data/10.22.2020/Data.shm
IndividualData/shaurya-data/11.15.2020/Data.shm
IndividualData/shaurya-data/11.17.2020/Data.shm
IndividualData/shaurya-data/11.1.2020/Data.shm
IndividualData/shaurya-data/11.18.2020/Data.shm

Loading Dataset ...
Loading File:  ../data/IndividualData/shaurya-data/10.22.2020/Data.shm
Loading File:  ../data/IndividualData/shaurya-data/11.15.2020/D

In [19]:
from collections import defaultdict

def test_models(winmin=1, stridesec = 5,names= ["wenkanw"],random_seed=1000, split_day=False):
    perf = defaultdict(list)
    meal_info = defaultdict(list)
    for name in names:
        person = name
        if split_day:
            meal_data = Person_MealsDataset(person_name= person, file_name = "test_files", winmin = winmin,stridesec = stridesec)

            # balance test set
            testset_labels = meal_data.labels
            test_indices = balance_data_indices(testset_labels,data_indices=[i for i in range(len(meal_data))] ,mode="under", shuffle=True,random_state = random_seed,replace= False)
            # get numpy dataset
            test_Data, test_Labels = meal_data.get_subset(test_indices)
        else:            
            meal_data = Person_MealsDataset(person_name= person, file_name = "all_files_list", winmin = winmin,stridesec = stridesec)
            samples,labels =  meal_data.data_indices, meal_data.labels
            # split train set and test set
            train_indices, test_indices = split_train_test_indices(X= [i for i in range(len(labels))],
                                                                            y = labels, test_size = 0.2,
                                                                           random_seed = random_seed)
            testset_labels = labels[test_indices]
            test_indices = balance_data_indices(testset_labels,data_indices= test_indices,mode="under", shuffle=True,random_state = random_seed,replace= False)
            test_Data, test_Labels = meal_data.get_subset(test_indices)
        if name !="CAD":
            meal_counts, min_counts,hour_counts, day_counts,total_hours = meal_data.get_mealdataset_info(person_name=name)
        else:
            # data from paper
            meal_counts, min_counts,hour_counts, day_counts,total_hours = 1063, 250*60,250, 354, 4680
        meal_info["dataset"].append(name)
        meal_info["Days"].append(day_counts)
        meal_info["Meal_Hours"].append(round(hour_counts,1)) 
#         meal_info["Meal_Minutes"].append(round(min_counts,1))
        meal_info["Meal_Counts"].append(meal_counts) 
        meal_info["Total_Hours"].append(total_hours) 
        
        perf["dataset"].append(name)
        perf["win(sec)"].append(winmin*60)
        
            
        
        
        from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, f1_score
        group_model_W  = tf.keras.models.load_model('../models/CAD_models/resnet_M_F_1.000000Min.h5')
        group_model_S  = tf.keras.models.load_model('../models/ActiModels/CAD_M_F_1.000000Min.h5')
        if name != "CAD":
            if split_day:
                individual_model = tf.keras.models.load_model('../models/'+ name+ '_models/resnet_split_day_M_F_1.000000Min.h5')
            else:    
                individual_model = tf.keras.models.load_model('../models/'+ name+ '_models/resnet_M_F_1.000000Min.h5')
        models = {"suffix":['Individual-Model','W-GroupModel',"S-GroupModel"],  "model":[individual_model,group_model_W,group_model_S]}
        
        for i in range(len(models["suffix"])):
            suffix = models["suffix"][i]
            model = models["model"][i]
            # if the dataset is CAD group dataset and model is individual model
            # we don't need to make prediction on that data
            if name == "CAD" and suffix =='Individual-Model':
                acc = None
                auc = None
                recall = None
            else:

                predictions = model.predict(x=test_Data)
                threshold = 0.5
                acc =  accuracy_score(test_Labels,predictions>=threshold)
                recall = recall_score(test_Labels,predictions>=threshold)
                f1 = f1_score(test_Labels,predictions>=threshold)
#                 auc = roc_auc_score(test_Labels,predictions>=threshold)

                acc = round(acc,5)
                recall = round(recall, 5)
#                 auc = round(auc,5)
                f1 = round(f1,5)
            
            print("Test Accuracy:", acc)
            print("Recall Accuracy:", recall)
            #print("AUC Score:", auc)
            perf["Acc: "+suffix].append(acc)
            perf["Recall: "+suffix].append(recall)
            perf["F1: "+suffix].append(f1)
            #perf["Auc: "+suffix].append(auc)

    meal_info = pd.DataFrame(meal_info)
    perf_df = pd.DataFrame(perf)
    return meal_info, perf_df

In [153]:
meal_info, performance = test_models(winmin=1, stridesec = 5,names= ["wenkanw","adam","lawler","shaurya","CAD"],random_seed=1000, split_day=False)

Loading Dataset ...
Loading File:  ../data/IndividualData/wenkanw-data/11-14-20/11-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-13-20/10-13-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-9-20/10-10-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-21-20/Dinner/Dinner.shm
Loading File:  ../data/IndividualData/wenkanw-data/9-17-20/9-17-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/9-29-20/9-29-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-22-20/Lunch/Lunch.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-21-20/lunch/lunch.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-9-20/10-9-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-18-20/11-18-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-14-20/10-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-12-20/10-12-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-16-20/11-16-20.shm
Loading File:  ../data/Indi

In [154]:
meal_info

,dataset,Days,Meal_Hours,Meal_Counts,Total_Hours
0,wenkanw,17,7,32,129
1,adam,20,4,36,168
2,lawler,23,25,61,173
3,shaurya,6,3,17,65
4,CAD,354,250,1063,4680


In [155]:
performance

,dataset,win(sec),Acc: Individual-Model,Recall: Individual-Model,F1: Individual-Model,Acc: W-GroupModel,Recall: W-GroupModel,F1: W-GroupModel,Acc: S-GroupModel,Recall: S-GroupModel,F1: S-GroupModel
0,wenkanw,60,0.92780,0.93141,0.92806,0.69449,0.46029,0.60106,0.74639,0.57310,0.69323
1,adam,60,0.95434,0.98353,0.95564,0.82784,0.71257,0.80541,0.79566,0.68413,0.77001
2,lawler,60,0.84309,0.79048,0.83438,0.65371,0.52882,0.60429,0.65416,0.58188,0.62722
3,shaurya,60,0.83569,0.75088,0.82046,0.66784,0.36926,0.52645,0.67933,0.43110,0.57344
4,CAD,60,NaN,NaN,0.57344,0.77677,0.67194,0.75063,0.75852,0.69825,0.74303


In [156]:
meal_info_split_day, performance_split_day = test_models(winmin=1, stridesec = 5,names= ["wenkanw","adam","lawler","shaurya","CAD"],random_seed=1000, split_day=True)

Loading Dataset ...
Loading File:  ../data/IndividualData/wenkanw-data/11-14-20/11-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-13-20/10-13-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-9-20/10-10-20.shm
Test Accuracy: 0.80493
Recall Accuracy: 0.66847
Test Accuracy: 0.62259
Recall Accuracy: 0.31611
Test Accuracy: 0.68042
Recall Accuracy: 0.45875
Loading Dataset ...
Loading File:  ../data/IndividualData/adam-data/12-08-2020/12-08-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-24-2020/11-24-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-30-2020/11-30-2020.shm
Loading File:  ../data/IndividualData/adam-data/12-09-2020/12-09-2020.shm
Test Accuracy: 0.92225
Recall Accuracy: 0.86722
Test Accuracy: 0.95036
Recall Accuracy: 0.94019
Test Accuracy: 0.9372
Recall Accuracy: 0.96651
Loading Dataset ...
Loading File:  ../data/IndividualData/lawler-data/10.19/10.19.shm
Loading File:  ../data/IndividualData/lawler-data/11.14/11.14.shm
Loading

In [157]:
meal_info_split_day

,dataset,Days,Meal_Hours,Meal_Counts,Total_Hours
0,wenkanw,17,7,32,129
1,adam,20,4,36,168
2,lawler,23,25,61,173
3,shaurya,6,3,17,65
4,CAD,354,250,1063,4680


In [158]:
performance_split_day

,dataset,win(sec),Acc: Individual-Model,Recall: Individual-Model,F1: Individual-Model,Acc: W-GroupModel,Recall: W-GroupModel,F1: W-GroupModel,Acc: S-GroupModel,Recall: S-GroupModel,F1: S-GroupModel
0,wenkanw,60,0.80493,0.66847,0.77411,0.62259,0.31611,0.45581,0.68042,0.45875,0.58940
1,adam,60,0.92225,0.86722,0.91772,0.95036,0.94019,0.94985,0.93720,0.96651,0.93899
2,lawler,60,0.77284,0.71965,0.76008,0.70088,0.65457,0.68635,0.68899,0.66708,0.68202
3,shaurya,60,0.48000,0.12667,0.19588,0.52500,0.10000,0.17391,0.54833,0.17333,0.27733
4,CAD,60,NaN,NaN,0.27733,0.77561,0.67266,0.74986,0.75316,0.69103,0.73681


In [142]:
meal_info_split_day, performance_split_day = test_models(winmin=1, stridesec = 5,names= ["wenkanw","adam","lawler","shaurya","CAD"],random_seed=1000, split_day=True)

Loading Dataset ...
Loading File:  ../data/IndividualData/wenkanw-data/11-14-20/11-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-13-20/10-13-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-9-20/10-10-20.shm
Test Accuracy: 0.80493
Recall Accuracy: 0.66847
AUC Score: 0.80493
Test Accuracy: 0.62259
Recall Accuracy: 0.31611
AUC Score: 0.62259
Test Accuracy: 0.68042
Recall Accuracy: 0.45875
AUC Score: 0.68042
Loading Dataset ...
Loading File:  ../data/IndividualData/adam-data/12-08-2020/12-08-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-24-2020/11-24-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-30-2020/11-30-2020.shm
Loading File:  ../data/IndividualData/adam-data/12-09-2020/12-09-2020.shm
Test Accuracy: 0.92225
Recall Accuracy: 0.86722
AUC Score: 0.92225
Test Accuracy: 0.95036
Recall Accuracy: 0.94019
AUC Score: 0.95036
Test Accuracy: 0.9372
Recall Accuracy: 0.96651
AUC Score: 0.9372
Loading Dataset ...
Loading File:  ../data/Ind

In [143]:
meal_info_split_day

,dataset,Days,Meal_Hours,Meal_Counts,Total_Hours
0,wenkanw,17,7,32,129
1,adam,20,4,36,168
2,lawler,23,25,61,173
3,shaurya,6,3,17,65
4,CAD,354,250,1063,4680


In [144]:
performance_split_day

,dataset,win(sec),Acc: Individual-Model,Recall: Individual-Model,Auc: Individual-Model,Acc: W-GroupModel,Recall: W-GroupModel,Auc: W-GroupModel,Acc: S-GroupModel,Recall: S-GroupModel,Auc: S-GroupModel
0,wenkanw,60,0.80493,0.66847,0.80493,0.62259,0.31611,0.62259,0.68042,0.45875,0.68042
1,adam,60,0.92225,0.86722,0.92225,0.95036,0.94019,0.95036,0.93720,0.96651,0.93720
2,lawler,60,0.77284,0.71965,0.77284,0.70088,0.65457,0.70088,0.68899,0.66708,0.68899
3,shaurya,60,0.48000,0.12667,0.48000,0.52500,0.10000,0.52500,0.54833,0.17333,0.54833
4,CAD,60,NaN,NaN,NaN,0.77561,0.67266,0.77561,0.75316,0.69103,0.75316


In [9]:
# Save model to pb file
# model_path = '../models/wenkanw_models/resnet_M_F_1.000000Min.h5'
# model = keras.models.load_model(model_path)
# model.save("./log/wenkanw_model2.pb")

INFO:tensorflow:Assets written to: ./log/wenkanw_model2.pb/assets


In [110]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
# time_series = np.random.random(10000)

# meal_data = Person_MealsDataset(person_name= "wenkanw", file_name = "test_files", winmin = 1,stridesec = 5)
meal_data = Person_MealsDataset(person_name= "wenkanw", file_name = "test_files", winmin = 1,stridesec = 5,smooth_flag = 0,
                 normalize_flag = 0)
time_series = meal_data.data[0].T

# Remove outliers
# for i in range(len(time_series)):
#     med = np.quantile(time_series[i],[0.5])
#     boundary = np.quantile(time_series[i],[0.01,0.99])
#     time_series[i][time_series[i]>boundary[1]] = med
#     time_series[i][time_series[i]<boundary[0]] = med
    
def f(offset,winsize,moment):
    if moment!= None:
        offset = int(moment)
    if offset < time_series.shape[1]-winsize:
        t = np.arange(start = offset, stop= offset+winsize, step=1)
        fig, ax = plt.subplots(3,1,figsize= (20,10))
        df1 = time_series[3, offset: offset+winsize]
        df2 = time_series[4, offset: offset+winsize]
        df3 = time_series[5, offset: offset+winsize]
        # set y-axis center at 0
        ax[0].set_ylim(-max(abs(df1)),max(abs(df1)))
        ax[1].set_ylim(-max(abs(df2)),max(abs(df2)))
        ax[2].set_ylim(-max(abs(df3)),max(abs(df3)))
        
        _ = sns.lineplot(t,df1 , ax =ax[0])
        _ = sns.lineplot(t, df2, ax =ax[1])
        _ = sns.lineplot(t, df3, ax =ax[2])
        
    
    return 
interact(f, offset=(0, time_series.shape[1]), winsize= [15*30], moment = '0');

Loading Dataset ...
Loading File:  ../data/IndividualData/wenkanw-data/10-14-20/10-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-18-20/11-18-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/9-17-20/9-17-20.shm


interactive(children=(IntSlider(value=258711, description='offset', max=517423), Dropdown(description='winsize…

In [80]:
# meal_data = Person_MealsDataset(person_name= "wenkanw", file_name = "test_files", winmin = 1,stridesec = 5,smooth_flag = 0,
#                  normalize_flag = 0)


Loading Dataset ...
Loading File:  ../data/IndividualData/wenkanw-data/10-14-20/10-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-18-20/11-18-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/9-17-20/9-17-20.shm


In [102]:
# df = meal_data.data[0].T
# num = 2
# np.max(df[num]), np.min(df[num]),len(df[df>30]),len(df[df<-20])
# num = 3
# np.quantile(df[num],[0,0.02,0.25,0.5,0.75,0.98,1])

array([-4.11059570e+02, -4.56250153e+01, -2.53175354e+00,  1.21917725e-02,
        2.71726990e+00,  5.33414972e+01,  6.80175781e+02])

In [20]:
meal_data = Person_MealsDataset(person_name= "wenkanw", file_name = "test_files", winmin = 1,stridesec = 5,smooth_flag = 1,
                 normalize_flag = 1)

Loading Dataset ...
Loading File:  ../data/IndividualData/wenkanw-data/10-14-20/10-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-18-20/11-18-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/9-17-20/9-17-20.shm


In [27]:
day  = 0
day1_data = meal_data.data_indices[meal_data.data_indices[:,0]==day]
day1_label = meal_data.labels[meal_data.data_indices[:,0]==day]
day1_data, day1_label

(array([[     0,      0,    900,  42966],
        [     0,     75,    975,  42966],
        [     0,    150,   1050,  42966],
        ...,
        [     0, 516300, 517200,  42966],
        [     0, 516375, 517275,  42966],
        [     0, 516450, 517350,  42966]]),
 array([0, 0, 0, ..., 0, 0, 0]))

In [28]:
len(meal_data.data_indices),len( meal_data.labels),set(day1_label)

(20266, 20266, {0, 1})

In [31]:
subdataset = meal_data.get_dataset(0, len(day1_data))

100%|██████████| 6887/6887 [00:16<00:00, 408.38it/s]


In [37]:
subdataset['data'].iloc[0]

tensor([[-0.8596,  1.9263,  1.1254, -0.0842,  0.1248, -3.1173],
        [-1.0928,  2.8200, -1.6128, -3.7340, -0.8169, -2.9935],
        [-0.4306,  4.1507, -1.5634,  0.4042,  0.2284, -1.2990],
        ...,
        [ 0.1852,  1.2797,  1.0220, -1.7421,  0.5912, -0.3239],
        [ 0.1933,  1.5122,  1.3155, -1.6786,  0.4341, -1.1580],
        [ 0.2658,  1.8572,  1.5431, -1.4450,  0.1537, -1.1859]])

In [35]:
sample, label = meal_data[0][0].numpy(),meal_data[0][1]

model = tf.keras.models.load_model('../models/'+ 'wenkanw'+ '_models/resnet_M_F_1.000000Min.h5')
s2 = np.expand_dims(sample,axis=0)
sample.shape, s2.shape, s2[0].all()==sample.all()
model(s2).numpy()[0][0],label

(4.8908533e-10, 0)

In [60]:
# for i in set(meal_data.data_indices[:,0]):
#     print(i)
# day=1
# np.where(meal_data.data_indices[:,0]==day)

(array([ 6887,  6888,  6889, ..., 13176, 13177, 13178]),)

# Hysteresis threshold

In [20]:

def hysteresis_threshold(model, data,start_threshold=0.8, end_threshold=0.3, winmin = 1,
                        stepsec=5, episode_min = 1.,t_pause = 900):
    """
    model: tensorflow model
    data:  This dataset must be the self-defined class of Person_MealsDataset  datasetset in my dataset.py/pytorch dataset without using shuffle. 
    Keep the order of dataset after extracting window samples!  You can also define your own dataset using class object to create the interface
    
    start_threshold: the high threshold of the beginning of segmentation
    
    end_threshold: the end threshold of the end of segmentation
    
    winmin: size of a window sample in unit of  minute
    
    stepsec: stride to move the window in unit of second / the number of second between two adjacent window samples
    
    episode_min: the minimum length of eating episode in unit of minute. If end of segmentation -start of segmentation < episode_min,
        then the episode will not be counted
    
    """
    result_ls = []
    
    
    days = set(data.data_indices[:,0])
    for day in days:
        # Select and Extract the data and labels of the corresponding day from the whole dataset
        sample_indices= np.where(data.data_indices[:,0]==day)[0]
        result = {'day':day, 'segment_start':[], 'segment_end':[],'proba':[],'predictions':np.zeros([len(sample_indices)]),'labels':[],"segment_count":0}
        
        # get the numpy array of samples and labels
        samples, labels = data.get_subset(sample_indices)
        probas = model(samples)
        state = 0
        start = 0
        end = 0 
        pause_counter = 0
        # one day data
        print("Day: ",day)
        for i in range(len(sample_indices)):
            #print("i:",i)
            #sample, label = data[i][0].numpy(),data[i][1]
            #sample = np.expand_dims(sample,axis=0)
            #proba = model(sample).numpy()[0][0]
            sample = samples[i]
            label = labels[i]
            proba = probas[i].numpy()[0]
            
            result['proba'].append(proba)
            result['labels'].append(label)
            
            if state ==0 and proba > start_threshold:
                state = 1
                start = i
            elif state == 1 and proba <end_threshold:
                state = 2
                end = i+1
                pause_counter = 0
            elif state ==2:
                if proba > start_threshold:
                    state = 1
                else:
                    pause_counter += stepsec
                    if pause_counter >= t_pause:
                        # convert time to second and check threshold
                        if (end-start)*stepsec >= episode_min*60:
                            # save data
                            result['segment_start'].append(start)
                            result['segment_end'].append(end)
                            result['segment_count'] += 1
                            result['predictions'][start:end] = 1
                            pass
                        end = 0
                        state = 0
        if state != 0:
            # if segment ended at the end of data
            if end != 0:
                result['segment_start'].append(start)
                result['segment_end'].append(end)
                result['predictions'][start:end] = 1
            else:
                result['segment_count'] -= 1  
            result['segment_count'] += 1
            
        result_ls.append(result)
        print("Segmentation Completed. ")
                            
    return pd.DataFrame(result_ls)

In [21]:
def print_results(result):
    from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
    total_preds = []
    total_labels = []
    result_df = {"Day":[],"Accuracy":[],"Recall":[],"AUC":[]}
    for i in range(len(result)):
        preds = result.iloc[i]['predictions']
        labels =  result.iloc[i]['labels']
        #print("Day: ", i,"Sample Num: ", len(preds) , "Prediction values: ",set(preds),"Segment Count: ",result.iloc[i]['segment_count'])

        acc = accuracy_score(labels,preds)
        rec = recall_score(labels,preds)
        auc = roc_auc_score(labels,preds)
        #print("Day-"+str(i), " Accuracy: ", acc, " Recall: ",rec,"Auc: ",auc)
        #print()
        result_df["Day"].append(str(i))
        result_df["Accuracy"].append(acc)
        result_df["Recall"].append(rec)
        result_df["AUC"].append(auc)
        total_preds.extend(preds)
        total_labels.extend(labels)

    acc= accuracy_score(total_labels,total_preds)
    rec = recall_score(total_labels,total_preds)
    auc = roc_auc_score(total_labels,total_preds)
    
    print("All Days: "," Accuracy: ", acc, " Recall: ",rec,"AUC",auc)
    result_df["Day"].append("All Days")
    result_df["Accuracy"].append(acc)
    result_df["Recall"].append(rec)
    result_df["AUC"].append(auc)
    return pd.DataFrame(result_df)


In [22]:
def find_segment(labels):
    start = []
    end = []
    for i in range(len(labels)):
        if labels[i]==1 and (i==0 or labels[i-1]==0):
            start.append(i)
        if labels[i]==1 and (i ==len(labels)-1 or  labels[i+1] ==0):
            end.append(i)
        
    return start, end

In [23]:
def get_episode_metrics(result, meal_data =None):
    from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
    total_preds = []
    total_labels = []
    perf = {"TPR":[],"FP/TP":[],"TP":[], "FP":[],"FN":[]}
    tpr = 0. 
    FP_TP = 0.
    TP = 0
    FP = 0
    FN = 0
    result_df = {"Day":[],"Accuracy":[],"Recall":[],"AUC":[]}
    for i in range(len(result)):
        preds = result.iloc[i]['predictions']
        labels =  result.iloc[i]['labels']
        
        for index in range(len(result.iloc[i]['segment_start'])):
            s = result.iloc[i]['segment_start'][index]
            e = result.iloc[i]['segment_end'][index]
            if sum(labels[s:e]) >0:
                # detect one meal
                TP += 1 #e-s + 1
            else:
                # detect wrongly
                FP += 1 #e- s + 1
        
        labels_start, labels_end =  find_segment(labels)
        for index in range(len(labels_start)):
            s = labels_start[index]
            e = labels_end[index]
            # missing meal
            if sum(preds[s:e]) == 0:
                FN += 1 #e - s + 1
            
            
                
    
    print("TP: ", TP, "FP: ", FP, "FN: ", FN)
    perf['TPR'].append(TP/(TP+FN))
    if TP ==0:
        perf['FP/TP'].append(None)
    else:
        perf['FP/TP'].append(FP/TP)
    perf["TP"].append(TP)
    perf["FP"].append(FP)
    perf["FN"].append(FN)
    result_df = pd.DataFrame(perf)
        
    return pd.DataFrame(result_df)


In [26]:
person = 'adam'
meal_data = Person_MealsDataset(person_name= person, file_name = "test_files", winmin = 1,stridesec = 5,smooth_flag = 1,
                 normalize_flag = 1)

# meal_data = Person_MealsDataset(person_name= person, file_name = "all_files_list", winmin = 1,stridesec = 5,smooth_flag = 1,
#                  normalize_flag = 1)
model = tf.keras.models.load_model('../models/'+ person + '_models/resnet_M_F_1.000000Min.h5')
# model = tf.keras.models.load_model('../models/CAD_models/resnet_M_F_1.000000Min.h5')
# model = tf.keras.models.load_model('../models/'+ person+ '_models/resnet_M_F_1.000000Min.h5')

threshold = {'wenkanw':[0.8, 0.3], 'adam':[0.8,0.3],'lawler':[0.8,0.3], 'shaurya':[0.8,0.3]}
# threshold = {'wenkanw':[0.9, 0.3], 'adam':[0.9,0.3],'lawler':[0.9,0.3], 'shaurya':[0.9,0.3]}
# threshold = {'wenkanw':[0.8, 0.4], 'adam':[0.8,0.4],'lawler':[0.8,0.4], 'shaurya':[0.8,0.4]}
high_th, low_th = threshold[person][0], threshold[person][1]
result = hysteresis_threshold(model, meal_data,start_threshold=high_th, end_threshold=low_th, winmin = 1, stepsec=5, episode_min = 1.)
episode_perf_df = get_episode_metrics(result)
episode_perf_df

Loading Dataset ...
Loading File:  ../data/IndividualData/adam-data/12-08-2020/12-08-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-24-2020/11-24-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-30-2020/11-30-2020.shm
Loading File:  ../data/IndividualData/adam-data/12-09-2020/12-09-2020.shm
Day:  0
Segmentation Completed. 
Day:  1
Segmentation Completed. 
Day:  2
Segmentation Completed. 
Day:  3
Segmentation Completed. 
TP:  9 FP:  5 FN:  0


,TPR,FP/TP,TP,FP,FN
0,1.0,0.555556,9,5,0


In [27]:
# meal_data.person_name #.get_mealdataset_info(person_name=person)

In [28]:
model = tf.keras.models.load_model('../models/CAD_models/resnet_M_F_1.000000Min.h5')

high_th, low_th = threshold[person][0], threshold[person][1]
result = hysteresis_threshold(model, meal_data,start_threshold=high_th, end_threshold=low_th, winmin = 1, stepsec=5, episode_min = 1.)
episode_perf_df = get_episode_metrics(result)
episode_perf_df

Day:  0
Segmentation Completed. 
Day:  1
Segmentation Completed. 
Day:  2
Segmentation Completed. 
Day:  3
Segmentation Completed. 
TP:  9 FP:  7 FN:  0


,TPR,FP/TP,TP,FP,FN
0,1.0,0.777778,9,7,0


In [133]:
# result

In [136]:
cnt = 0
cnt2 = 0
for i in range(len(result)):
    labels =  result.iloc[i]['labels']
    labels_start, labels_end =  find_segment(labels)
    cnt += len(labels_start)
    cnt2 += len(labels_end)
print("Count: ", cnt,cnt2)
    

Count:  60 60


In [126]:
person = 'adam'
meal_data = Person_MealsDataset(person_name= person, file_name = "test_files", winmin = 1,stridesec = 5,smooth_flag = 1,
                 normalize_flag = 1)
model = tf.keras.models.load_model('../models/'+ person + '_models/resnet_M_F_1.000000Min.h5')

threshold = {'wenkanw':[0.8, 0.3], 'adam':[0.8,0.3],'lawler':[0.8,0.3], 'shaurya':[0.8,0.3]}
# threshold = {'wenkanw':[0.9, 0.3], 'adam':[0.9,0.3],'lawler':[0.9,0.3], 'shaurya':[0.9,0.3]}
# threshold = {'wenkanw':[0.8, 0.4], 'adam':[0.8,0.4],'lawler':[0.8,0.4], 'shaurya':[0.8,0.4]}
high_th, low_th = threshold[person][0], threshold[person][1]
result = hysteresis_threshold(model, meal_data,start_threshold=high_th, end_threshold=low_th, winmin = 1, stepsec=5, episode_min = 1.)
episode_perf_df1 = get_episode_metrics(result)
episode_perf_df1

Loading Dataset ...
Loading File:  ../data/IndividualData/adam-data/12-08-2020/12-08-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-24-2020/11-24-2020.shm
Loading File:  ../data/IndividualData/adam-data/11-30-2020/11-30-2020.shm
Loading File:  ../data/IndividualData/adam-data/12-09-2020/12-09-2020.shm
Day:  0
Segmentation Completed. 
Day:  1
Segmentation Completed. 
Day:  2
Segmentation Completed. 
Day:  3
Segmentation Completed. 
TP:  9 FP:  5 FN:  0


,TPR,FP/TP,TP,FP,FN
0,1.0,0.555556,9,5,0


In [127]:
model = tf.keras.models.load_model('../models/CAD_models/resnet_M_F_1.000000Min.h5')
high_th, low_th = threshold[person][0], threshold[person][1]
result = hysteresis_threshold(model, meal_data,start_threshold=high_th, end_threshold=low_th, winmin = 1, stepsec=5, episode_min = 1.)
episode_perf_df2 = get_episode_metrics(result)
episode_perf_df2

Day:  0
Segmentation Completed. 
Day:  1
Segmentation Completed. 
Day:  2
Segmentation Completed. 
Day:  3
Segmentation Completed. 
TP:  9 FP:  7 FN:  0


,TPR,FP/TP,TP,FP,FN
0,1.0,0.777778,9,7,0


In [122]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
# time_series = np.random.random(10000)

# meal_data = Person_MealsDataset(person_name= "wenkanw", file_name = "test_files", winmin = 1,stridesec = 5)
# meal_data = Person_MealsDataset(person_name= "wenkanw", file_name = "test_files", winmin = 1,stridesec = 5,smooth_flag = 0,
#                  normalize_flag = 0)


def f(offset,winsize,day):
    stride  = 5 *15 # 5 seconds between two adjacent labels/window samples
    
    day = int(day)
    proba = np.array(result.proba.iloc[day])
    preds = np.array(result.predictions.iloc[day])
    labels = np.array(result.labels.iloc[day])
    if winsize == -1:
        winsize = len(labels)
    
    if offset == len(labels)-winsize:
        offset = len(labels)-winsize
        
    t = np.arange(start = offset, stop= offset+winsize, step=1)
    fig, ax = plt.subplots(3,1,figsize= (20,12))
    df1 = proba[offset:offset+winsize]
    df2 = np.array(preds[offset:offset+winsize]) #*10-5
    df3 = np.array(labels[offset:offset+winsize])#*10-5
    x1= sns.lineplot(x=t, y=df1 , ax =ax[0],color= 'grey',label="Possibility")
    x2 = sns.lineplot(t,df2 , ax =ax[1],color='g', linewidth=1.5,label="Prediction(Eat)")
    x3 = sns.lineplot(t,df3 , ax =ax[2],color='b', linewidth=1.5, label="Label(Eat)")
        
    ax[0].fill_between( t, df1, 
                interpolate=True, color='grey')
    
    ax[0].fill_between(t, df3, where=(df3==1), 
                interpolate=True, color='blue')
    ax[0].fill_between( t,df2, where=(df2==1), 
                interpolate=True, color='green')
    ax[0].set_ylim(0,1)
    print(offset,len(labels)-winsize )
    if offset >= len(labels)-winsize:
        title_txt ="Day: "+ str(day) + " Whole Day samples: "+str(len(labels)) +" . " + "Sample plotted: "+str(winsize)+". "
    else:
        title_txt = "Day: "+ str(day) + "Samples from "+str(offset) +"~" + str((offset+winsize)) +". "+ "Number of Sample plotted: "+str(winsize)+". "
    ax[0].set_title(title_txt)
    ax[2].set_xlabel("index of sample")
    ax[0].set_ylabel("Possibility")
    ax[1].set_ylabel("Predictions")
    ax[2].set_ylabel("Ground Truch")
    
    ax[0].legend(["Possibility"],loc='upper left')
    ax[1].legend(["1: Eat, 0:Other"],loc='upper left')
    ax[2].legend(["1: Eat, 0:Other"],loc='upper left')
        
    return 
style = {'description_width': 'initial'}
bound_text = widgets.BoundedIntText(
    value=0,
    min=0,
    max=len(result.proba.iloc[0]),
    step=1,
    description='offset:[0~'+ str(len(result.proba.iloc[0]))+"]",
    disabled=False,
    style = {'description_width': 'initial'}
)
slidebar= widgets.IntSlider(min=0, max=len(result.proba.iloc[0]), step=1, value=0)
winsize = widgets.Dropdown(
    options=[-1, 1000],
    value=-1,
    description='samples plotted',
    disabled=False,
    style = style
)
interact(f, offset=bound_text, winsize= winsize, day = [i for i in range(len(result))]);

interactive(children=(BoundedIntText(value=0, description='offset:[0~8549]', max=8549, style=DescriptionStyle(…

In [123]:
from collections import defaultdict

def test_models_hysteresis_threshold(winmin=1, stridesec = 5,names= ["wenkanw"],random_seed=1000, test_day=True,thresholds = None,episode_min=1.,high_th = 0.8,low_th= 0.3):
    perf = defaultdict(list)
    for name in names:
        person = name
        if test_day:
            meal_data = Person_MealsDataset(person_name= person, file_name = "test_files", winmin = winmin,stridesec = stridesec,smooth_flag = 1,
                         normalize_flag = 1)
        else:
            meal_data = Person_MealsDataset(person_name= person, file_name = "all_files_list", winmin = winmin,stridesec = stridesec,smooth_flag = 1,
                         normalize_flag = 1)
            
#         model = tf.keras.models.load_model('../models/'+ person+ '_models/resnet_M_F_1.000000Min.h5')
        
        perf["dataset"].append(name)
        perf["win(sec)"].append(winmin*60)
        
            
        from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
        group_model_W  = tf.keras.models.load_model('../models/CAD_models/resnet_M_F_1.000000Min.h5')
        group_model_S  = tf.keras.models.load_model('../models/ActiModels/CAD_M_F_1.000000Min.h5')
        individual_model = tf.keras.models.load_model('../models/'+ name+ '_models/resnet_M_F_1.000000Min.h5')
        models = {"suffix":['Individual-Model','W-GroupModel',"S-GroupModel"],  "model":[individual_model,group_model_W,group_model_S]}
        
        for i in range(len(models["suffix"])):
            suffix = models["suffix"][i]
            model = models["model"][i]
            # if the dataset is CAD group dataset and model is individual model
            # we don't need to make prediction on that data
            if name == "CAD" and suffix =='Individual-Model':
                acc = None
                auc = None
                recall = None
            else:
                if thresholds != None and person in thresholds.keys():
                    high_th, low_th = thresholds[person][0], thresholds[person][1]
                
                    
                result = hysteresis_threshold(model, meal_data,start_threshold=high_th, end_threshold=low_th, winmin = winmin, stepsec=stridesec, episode_min = episode_min)
                episode_perf_df = print_results(result)
                
                acc = episode_perf_df.iloc[-1]["Accuracy"]
                recall = episode_perf_df.iloc[-1]["Recall"]
                auc = episode_perf_df.iloc[-1]["AUC"]
                
                acc = round(acc,5)
                recall = round(recall, 5)
                auc = round(auc,5)
            
            print("Test Accuracy:", acc)
            print("Recall Accuracy:", recall)
            print("AUC Score:", auc)
            perf["Acc: "+suffix].append(acc)
            perf["Recall: "+suffix].append(recall)
            perf["Auc: "+suffix].append(auc)

    
    perf_df = pd.DataFrame(perf)
    return  perf_df

In [116]:
threshold = {'wenkanw':[0.95, 0.45], 'adam':[0.95,0.45],'lawler':[0.95,0.45], 'shaurya':[0.95,0.45],'CAD':[0.95,0.45]}
perf = test_models_hysteresis_threshold(names= ["wenkanw","adam","lawler","shaurya","CAD"],random_seed=1000, test_day=True, thresholds= threshold)

Loading Dataset ...
Loading File:  ../data/IndividualData/wenkanw-data/10-14-20/10-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-18-20/11-18-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/9-17-20/9-17-20.shm
Day:  0
Segmentation Completed. 
Day:  1
Segmentation Completed. 
Day:  2
Segmentation Completed. 
All Days:   Accuracy:  0.9343234974834699  Recall:  0.9673659673659674 AUC 0.9501143521908155
Test Accuracy: 0.93432
Recall Accuracy: 0.96737
AUC Score: 0.95011
Day:  0
Segmentation Completed. 
Day:  1
Segmentation Completed. 
Day:  2
Segmentation Completed. 
All Days:   Accuracy:  0.9585512681338202  Recall:  0.039627039627039624 AUC 0.5194013186593566
Test Accuracy: 0.95855
Recall Accuracy: 0.03963
AUC Score: 0.5194
Day:  0
Segmentation Completed. 
Day:  1
Segmentation Completed. 
Day:  2
Segmentation Completed. 
All Days:   Accuracy:  0.9610184545544261  Recall:  0.1247086247086247 AUC 0.5613495720410395
Test Accuracy: 0.96102
Recall Accuracy: 0.12471
A

In [117]:
perf

,dataset,win(sec),Acc: Individual-Model,Recall: Individual-Model,Auc: Individual-Model,Acc: W-GroupModel,Recall: W-GroupModel,Auc: W-GroupModel,Acc: S-GroupModel,Recall: S-GroupModel,Auc: S-GroupModel
0,wenkanw,60,0.93432,0.96737,0.95011,0.95855,0.03963,0.51940,0.96102,0.12471,0.56135
1,adam,60,0.98204,0.85029,0.91801,0.97776,0.35125,0.67326,0.97860,0.33973,0.66809
2,lawler,60,0.95949,0.53818,0.75949,0.95186,0.00000,0.50000,0.95089,0.01297,0.50565
3,shaurya,60,0.97710,0.52542,0.75982,0.96524,0.17797,0.58652,0.96720,0.53107,0.75740
4,CAD,60,NaN,NaN,NaN,0.95452,0.49077,0.73608,0.94896,0.47685,0.72658


In [24]:
# from __future__ import print_function
# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
# def f(x):
#     return x
# interact(f, x=True);

In [25]:
# ! jupyter --version